In [7]:
# %%writefile alpha_base10.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
import tools.Pretreat_Tools as pretreat


def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'+ str(y.index[0])
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se

def excute_for_multidates(data, func, level=0, **pramas):
    return data.groupby(level=level, group_keys=False).apply(func,**pramas)



In [8]:
from tools.Cacher import (CACHE_TYPE, load_cache,load_caches_adv)
import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher

stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
# stock_df = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()
ret_fs = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs = (ret_fs['high']+ret_fs['low'])/2 * 0.01
ret_fs_daily = ret_fs/252
ret_fs_daily

date
2002-07    0.000114
2002-08    0.000113
2002-09    0.000122
2002-10    0.000122
2002-11    0.000132
             ...   
2022-09    0.000108
2022-10    0.000109
2022-11    0.000111
2022-12    0.000116
2023-01    0.000116
Length: 247, dtype: float64

In [10]:
# codes = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)
# 

ret_t = smpl.get_current_return(stock_df,'close')
ret_t_excess = ret_t.groupby(pd.Grouper(level='date', freq='1M')).apply(lambda x:x-ret_fs_daily.get(x.index[0][0].strftime('%Y-%m')))

def cal_ret_markey_daily(stock_daily_df,ret_t_excess_daily):
    ##  不取对数有时候有精度问题,权重不是精确1
    market_value_log = np.log(stock_daily_df['totalCapital'])
    market_value = market_value_log.sum()
    weight = market_value_log / market_value
    ret_market_f = (ret_t_excess_daily * weight).sum()
    return ret_market_f

ret_market_t = ret_t_excess.groupby(level=0, group_keys=False).apply(lambda ret: cal_ret_markey_daily(stock_df.loc[ret.index[0][0]],ret))

window=252
half_life_window = 63
half_life = list(map(lambda n:0.5**(n/half_life_window),range(0,window)))
half_life_weight = half_life/np.sum(half_life)


def beta():
    pass

# ret_t.loc[('2017-01',slice(None))]
# data.groupby(level=level, group_keys=False).apply(func,**pramas)



# model = linear_model.LinearRegression(fit_intercept=True).fit(X, factor,sample_weight=half_life_weight)
# neutralize_factor = factor - model.predict(X)

In [16]:
# ret_t_excess.index#.loc[(slice(None),'000001')].iloc[1:253]
ret_t_excess.dropna()

date        code  
2017-01-03  000001         NaN
            000002         NaN
            000006         NaN
            000007         NaN
            000008         NaN
                        ...   
2021-12-31  605580   -0.000113
            605588    0.000140
            605589   -0.031474
            605598   -0.021011
            605599    0.026584
Name: ret, Length: 3247889, dtype: float64

In [6]:
ret_market_t.iloc[1:252]

NameError: name 'ret_market_t' is not defined

In [114]:
# model = linear_model.LinearRegression(fit_intercept=True).fit(ret_t_excess.loc[(slice(None),'000006')].iloc[1:253].values.reshape(-1, 1), ret_market_t.iloc[1:253].values.reshape(-1, 1),sample_weight=half_life_weight)
# float(model.coef_),float(model.intercept_)

model = linear_model.LinearRegression(fit_intercept=True)
res_tmp = []
def reg(ret_t_ex):
    # print()
    # assert False,None
    res = model.fit(ret_t_ex.values.reshape(-1, 1), ret_market_t[ret_t_ex.index.get_level_values(0)].values.reshape(-1, 1),sample_weight=half_life_weight)
    res_tmp.append({'date':ret_t_ex.index[-1][0],'code':ret_t_ex.index[-1][1], 'beta':float(res.coef_), 'alpha':float(res.intercept_)})
    return 0
# ret_t_excess.dropna().loc[(slice(None),'000001')].rolling(window).apply(reg)
ret_t_excess.loc[(slice(None),['000001','000002'])].dropna().groupby(level=1,group_keys=False).apply(lambda x:x.rolling(window).apply(reg))

res_final = pd.DataFrame(res_tmp)
res_final.set_index(['date', 'code'], inplace=True)
res_final = res_final.sort_index()
pd.DataFrame(res_final['beta']).reset_index().to_feather('./beta_camp.feather')
pd.DataFrame(res_final['alpha']).reset_index().to_feather('./alpha_camp.feather')